In [ ]:
import os
import pandas as pd
import numpy as np

# === Paramètres ===
DATA_DIR = "data"           # Dossier contenant vos CSV sectoriels
TRADING_DAYS = 252          # Standard pour annualiser (252 jours de bourse/an)
MAX_MISSING_PCT = 0.1       # Seuil : on rejette une action si > 10% de données manquent

def get_market_data():
    """
    Charge les CSV, nettoie les données, calcule les rendements
    et génère les matrices pour l'optimisation de Markowitz.
    
    Returns:
        mu (pd.Series): Rendements moyens annualisés attendus.
        Sigma (pd.DataFrame): Matrice de covariance annualisée.
        returns (pd.DataFrame): Historique complet des rendements quotidiens.
    """
    if not os.path.exists(DATA_DIR):
        raise FileNotFoundError(f"Le dossier '{DATA_DIR}' n'existe pas. Lancez download.py d'abord.")

    # 1. Chargement et fusion des fichiers CSV
    sector_files = [f for f in os.listdir(DATA_DIR) if f.endswith(".csv")]
    
    if not sector_files:
        raise ValueError("Aucun fichier CSV trouvé dans le dossier data/.")

    print(f"Chargement de {len(sector_files)} fichiers sectoriels...")
    
    df_list = []
    for file in sector_files:
        path = os.path.join(DATA_DIR, file)
        # On charge en indexant par la Date
        df_sector = pd.read_csv(path, index_col="Date", parse_dates=True)
        df_list.append(df_sector)
    
    # Fusion en un seul gros DataFrame (alignement sur les dates)
    prices = pd.concat(df_list, axis=1)
    print(f"Dimensions brutes : {prices.shape[1]} actifs sur {prices.shape[0]} jours.")

    # 2. Nettoyage des données (Crucial pour éviter les erreurs de matrice)
    
    # a. Supprimer les actifs qui ont trop de NaNs (ex: entrés en bourse récemment)
    missing_ratios = prices.isna().mean()
    to_drop = missing_ratios[missing_ratios > MAX_MISSING_PCT].index
    prices = prices.drop(columns=to_drop)
    if len(to_drop) > 0:
        print(f"Actifs supprimés (trop de données manquantes) : {len(to_drop)}")

    # b. Remplir les petits trous (Forward Fill) puis supprimer les lignes restantes vides
    prices = prices.ffill().dropna()

    # 3. Calcul des Rendements (Returns)
    # Formule : R_t = (P_t / P_{t-1}) - 1
    returns = prices.pct_change().dropna()

    print(f"Univers final : {returns.shape[1]} actifs valides sur {returns.shape[0]} jours.")

    # 4. Calcul des inputs de Markowitz (Annualisation)
    # mu = Moyenne journalière * 252
    mu = returns.mean() * TRADING_DAYS
    
    # Sigma = Covariance journalière * 252
    Sigma = returns.cov() * TRADING_DAYS

    return mu, Sigma, returns

# === Bloc de test ===
if __name__ == "__main__":
    try:
        mu, Sigma, returns = get_market_data()
        
        print("\n--- Aperçu des données ---")
        print("Rendements moyens (Top 5) :")
        print(mu.head())
        print("\nMatrice de Covariance (taille) :", Sigma.shape)
        
        # Petit check mathématique : La matrice doit être symétrique
        is_symmetric = np.allclose(Sigma, Sigma.T)
        print(f"Matrice symétrique ? {is_symmetric}")
        
    except Exception as e:
        print(f"Erreur : {e}")

Chargement de 11 fichiers sectoriels...
Dimensions brutes : 197 actifs sur 1305 jours.
Actifs supprimés (trop de données manquantes) : 1
Univers final : 196 actifs valides sur 1248 jours.

--- Aperçu des données ---
Rendements moyens (Top 5) :
GOOGL    0.295471
META     0.367013
DIS      0.084867
NFLX     0.300734
VZ       0.014609
dtype: float64

Matrice de Covariance (taille) : (196, 196)
Matrice symétrique ? True


In [2]:
# === Configuration ===
FINAL_DATA_PATH = "data/returns_final.csv"
TRADING_DAYS = 252

# === Logique de chargement intelligent ===
if os.path.exists(FINAL_DATA_PATH):
    print("✅ Fichier de données nettoyées trouvé. Chargement rapide...")
    # On charge le CSV déjà prêt
    returns = pd.read_csv(FINAL_DATA_PATH, index_col="Date", parse_dates=True)
    
    # On recalcule juste mu et Sigma (c'est instantané)
    mu = returns.mean() * TRADING_DAYS
    Sigma = returns.cov() * TRADING_DAYS
    
    print(f"Données chargées : {returns.shape[1]} actifs sur {returns.shape[0]} dates.")

else:
    print("⚠️ Fichier non trouvé. Lancement du traitement complet (peut être long)...")
    # On lance la fonction lourde définie précédemment
    mu, Sigma, returns = get_market_data()
    
    # === SAUVEGARDE ICI ===
    print(f"💾 Sauvegarde des données dans {FINAL_DATA_PATH}...")
    returns.to_csv(FINAL_DATA_PATH)

# === Vérification rapide ===
print("\n--- Prêt pour l'optimisation ---")
print("Exemple de mu (Rendement annuel attendu) :")
print(mu.head())

⚠️ Fichier non trouvé. Lancement du traitement complet (peut être long)...
Chargement de 11 fichiers sectoriels...
Dimensions brutes : 197 actifs sur 1305 jours.
Actifs supprimés (trop de données manquantes) : 1
Univers final : 196 actifs valides sur 1248 jours.
💾 Sauvegarde des données dans data/returns_final.csv...

--- Prêt pour l'optimisation ---
Exemple de mu (Rendement annuel attendu) :
GOOGL    0.295471
META     0.367013
DIS      0.084867
NFLX     0.300734
VZ       0.014609
dtype: float64
